In [1]:
import ee
import geemap
import os
import datetime as dt
from datetime import datetime, timedelta
import numpy as np
from functions_floes import *
import earthaccess
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import geemap
import xarray as xr
import h5netcdf
import math

In [2]:
#!pip install ffmpeg-python

In [4]:
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

Enter verification code:  4/1ATx3LY6qA7wRCh4QxrC1cf3lJsmHlL6vnSQzE3Rf8i4YOoMAEloMOrce5Xg



Successfully saved authorization token.


In [4]:
region = ee.Geometry.BBox(-51, -78.5, -41, -76.5) # Landfast ice in the Weddell Sea

# Mukund's ROI
bbox = [-10, 72.6, 12, 79.2]
# bbox = [-176.3, -76.3, -153.9, -74.4]
region = ee.Geometry.BBox(bbox[0], bbox[1], bbox[2], bbox[3]) #ee.Geometry.Rectangle(bbox)
point = ee.Geometry.Point([np.mean([bbox[0], bbox[2]]), np.mean([bbox[1], bbox[3]])]).buffer(20000)
# Approximately (72.6 to 79.2 N) and (-10 to 12 E) April to May 2023
# Approximately (-76.3 to -74.4 S) and (-176.3 to -153.9 W) November - December 2023

# Grounded icebergs ROI

sub = "W" #"EW"

if sub == "EW":
    point = [-45.14, -76.6] # South Weddell
    offset = [3, 0.6]
elif sub == "W":
    point = [-58.4,-71.23] # West Weddell
    offset = [1.75, 0.60]
else:
    print("SET SUBAREA!! (E or EW)")

bbox = [point[0]-offset[0], point[1]-offset[1], point[0]+offset[0], point[1]+offset[1]]
# region = ee.Geometry.Point(point).buffer(50000)
region = ee.Geometry.BBox(bbox[0], bbox[1], bbox[2], bbox[3])

# region = ee.Geometry.Polygon([
#     [-51, -78.5], [-51, -76.5], [-41, -76.5], [-41, -78.5], [-51, -78.5]
# ])

# point = ee.Geometry.Point([-46, -77.5])

collection = ee.ImageCollection("COPERNICUS/S1_GRD")\
.filter(ee.Filter.eq('transmitterReceiverPolarisation', ["HH"]))\
.filterBounds(region)
# .filter(ee.Filter.Or(ee.Filter.eq('relativeOrbitNumber_start', 5), ee.Filter.eq('relativeOrbitNumber_start', 34),
#         ee.Filter.eq('relativeOrbitNumber_start', 136)))

# 2016: 165
# 2017: 5

### Generate GIF time series

In [4]:
## One cell for all images


# Define arguments for animation function parameters.
video_args = {
    "dimensions": 600,
    "region": region,
    "framesPerSecond": 2,
    "bands": ["HH"],
    "crs":'EPSG:3857',
    "min": -25,
    "max": -5,
    "gamma": 1
    
}

for year in [2020]:
    for month in range(1, 13):
        start_date = f"{year}-{str(month).zfill(2)}-01"
        if month == 12:
            end_date = f"{year}-{str(month+1).zfill(2)}-01"
        else:
            end_date = f"{year+1}-01-01"

        if year >= 2017:
            collection = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(region)\
            .filter(ee.Filter.eq('transmitterReceiverPolarisation', ["HH"]))
            # .filter(ee.Filter.Or(ee.Filter.eq('relativeOrbitNumber_start', 5), ee.Filter.eq('relativeOrbitNumber_start', 34),
            #         ee.Filter.eq('relativeOrbitNumber_start', 136)))
        else:
            collection = ee.ImageCollection("COPERNICUS/S1_GRD")\
            .filter(ee.Filter.eq('transmitterReceiverPolarisation', ["HH"]))\
            .filterBounds(region)
            # .filter(ee.Filter.Or(
            #                     ee.Filter.eq('relativeOrbitNumber_start', 169), ee.Filter.eq('relativeOrbitNumber_start', 64)) )

            # ee.Filter.eq('relativeOrbitNumber_start', 5), ee.Filter.eq('relativeOrbitNumber_start', 165),

        images = geemap.create_timeseries(
            collection, start_date, end_date, region, frequency="day", reducer="median"
        )
        
        s = images.aggregate_array('system:time_start').getInfo()
        days = []
        for sec in s:
            d = dt.datetime(1970, 1, 1) + dt.timedelta(seconds = sec / 1000)
            days.append(d.strftime("%Y-%m-%d"))
        
        print(start_date, end_date, images.size().getInfo())

        work_dir = "D:\\Floes\\S1_ground"
        out_gif = os.path.join(work_dir, f"S1_{start_date}_{end_date}.gif")
        geemap.download_ee_video(images, video_args, out_gif)

        texted_gif = os.path.join(work_dir, f"S1_{start_date}_{end_date}_text.gif")
        geemap.add_text_to_gif(
            out_gif,
            texted_gif,
            xy=("3%", "5%"),
            text_sequence=days,
            font_size=20,
            duration=800,
            font_color="#ffffff",
            add_progress_bar=False,
        )

        print("====================================================")

2022-02-01 2022-03-30 41
Generating URL...
Please wait ...
The GIF image has been saved to: D:\Floes\S1_ground\S1_2022-02-01_2022-03-30.gif


## Export all images into GEOTIFF Files (saved in Google Drive)

In [5]:
# calculate coverage area of image to roi
def coverage(img, tol = 1000):
    overlap = img.geometry().intersection(region, tol)
    ratio = overlap.area(tol).divide(region.area(tol))
    return ratio.getInfo()

In [29]:
## Export images to TIFF files
# Export the image, specifying the CRS, transform, and region.

for year in [2023]:
    for month in range(1, 13):
        start_date = f"{year}-{str(month).zfill(2)}-01"
        if month == 12:
            end_date = f"{year+1}-01-01"
        else:
            end_date = f"{year}-{str(month+1).zfill(2)}-01"
            
        collection = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(region)\
        .filterDate(start_date, end_date)\
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'))\
        .filter(ee.Filter.eq('instrumentMode', 'EW'))

        S1_ids = collection.aggregate_array('system:id').getInfo()

        print(start_date, end_date, len(S1_ids))

        for i in S1_ids:
            
            img = ee.Image(i) #.select('HH') #.divide(ee.Image(i).select('angle')).clip(region)

            # Scale reflectance bands and overwrite the original bands.
            HH_norm = img.select('HH').divide(img.select('angle')).rename('HH_norm')
            img2 = img.addBands(HH_norm, overwrite=True).select('HH_norm')
            
            projection = img2.projection().getInfo()
            ratio = coverage(img2)

            description = i.split("/")[-1][:32]
            description = description.replace(description[:16], "S1A_W_GRDH_1SSH")
            
            if ratio > 0.9:
                print(description)
                task = ee.batch.Export.image.toDrive(
                    image=img2,
                    description=description,
                    folder="Grounded",
                    scale=50,
                    maxPixels = 1e12,
                    crs='EPSG:32721', #projection['crs'],
                    # crsTransform=projection['transform'],
                    region=region,
                )            
                task.start()

2023-01-01 2023-02-01 15
S1A_W_GRDH_1SSH_20230104T001430
S1A_W_GRDH_1SSH_20230105T072225
S1A_W_GRDH_1SSH_20230112T071413
S1A_W_GRDH_1SSH_20230116T001429
S1A_W_GRDH_1SSH_20230117T072224
S1A_W_GRDH_1SSH_20230124T004706
S1A_W_GRDH_1SSH_20230124T071413
S1A_W_GRDH_1SSH_20230128T001429
S1A_W_GRDH_1SSH_20230129T072225
2023-02-01 2023-03-01 15
S1A_W_GRDH_1SSH_20230205T004706
S1A_W_GRDH_1SSH_20230205T071413
S1A_W_GRDH_1SSH_20230210T072224
S1A_W_GRDH_1SSH_20230217T071412
S1A_W_GRDH_1SSH_20230221T001428
S1A_W_GRDH_1SSH_20230222T072223
2023-03-01 2023-04-01 17
S1A_W_GRDH_1SSH_20230301T004706
S1A_W_GRDH_1SSH_20230301T071413
S1A_W_GRDH_1SSH_20230305T001429
S1A_W_GRDH_1SSH_20230306T072224
S1A_W_GRDH_1SSH_20230313T071412
S1A_W_GRDH_1SSH_20230317T001428
S1A_W_GRDH_1SSH_20230318T072224
S1A_W_GRDH_1SSH_20230320T003900
S1A_W_GRDH_1SSH_20230325T071413
S1A_W_GRDH_1SSH_20230329T001429
S1A_W_GRDH_1SSH_20230330T072224
2023-04-01 2023-05-01 17
S1A_W_GRDH_1SSH_20230401T003900
S1A_W_GRDH_1SSH_20230406T071413
S1A_

In [107]:
year = 2020
month = 11
start_date = f"{year}-{str(month).zfill(2)}-01"
if month == 12:
    end_date = f"{year+1}-01-01"
else:
    end_date = f"{year}-{str(month+1).zfill(2)}-01"    

print(start_date, end_date)

collection = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(region)\
.filterDate(start_date, end_date)\
.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'))

S1_ids = collection.aggregate_array('system:id').getInfo()

for i in S1_ids[1:]:
    img = ee.Image(i) #.select('HH') #.divide(ee.Image(i).select('angle')).clip(region)
    print(i)
    # Scale reflectance bands and overwrite the original bands.
    HH_norm = img.select('HH').divide(img.select('angle')).rename('HH_norm')
    img2 = img.addBands(HH_norm, overwrite=True)
    
    # projection = img.projection().getInfo()
    ratio = coverage(img2)

    if ratio > 0.9:
        break

2020-11-01 2020-12-01
COPERNICUS/S1_GRD/S1A_EW_GRDM_1SSH_20201104T072216_20201104T072316_035095_041876_5774


In [108]:
# i = S1_ids[18]

img2 = img2.select('HH_norm').clip(region)
print(img2.get('relativeOrbitNumber_start').getInfo(), img2.get('sliceNumber').getInfo())
print(img2.get('system:asset_size').getInfo())
print(coverage(img2))
projection = img2.projection().getInfo()

Map = geemap.Map()

Map.addLayer(img2, {'min':-1, 'max': 1})
# Map.addLayer(region, {})
Map.centerObject(img, 7)
Map

23 5
497375860
0.999000670822082


Map(center=[-70.64488613509872, -57.80326979820369], controls=(WidgetControl(options=['position', 'transparent…

## Load ICESat-2 (via Earthacess)

In [6]:
import earthaccess
auth = earthaccess.login()

In [5]:
Query = earthaccess.collection_query().keyword('ICESat-2')

Query = earthaccess.collection_query().keyword('ATL10').cloud_hosted(True)
print(Query.hits())
collections = Query.fields(['ShortName', 'Version']).get(25)
print(collections)

3
[{
  "meta": {
    "concept-id": "C2613553243-NSIDC_CPRD",
    "granule-count": 49199,
    "provider-id": "NSIDC_CPRD"
  },
  "umm": {
    "ShortName": "ATL10",
    "Version": "006"
  }
}, {
  "meta": {
    "concept-id": "C2753295020-NSIDC_CPRD",
    "granule-count": 136,
    "provider-id": "NSIDC_CPRD"
  },
  "umm": {
    "ShortName": "ATL20",
    "Version": "004"
  }
}, {
  "meta": {
    "concept-id": "C2753316241-NSIDC_CPRD",
    "granule-count": 136,
    "provider-id": "NSIDC_CPRD"
  },
  "umm": {
    "ShortName": "ATL21",
    "Version": "003"
  }
}]


In [8]:
for year in [2021, 2022, 2023]:
    
    img_files = glob.glob(f"D:\\Floes\\S1_ground\\S1A_{sub}_GRDH_1SSH_{year}*.tif")
    
    Query = earthaccess.granule_query().concept_id('C2613553243-NSIDC_CPRD')\
    .temporal(f'{year}-01-01', f'{year+1}-01-01').bounding_box(bbox[0], bbox[1], bbox[2], bbox[3])
    
    N = Query.hits()
    granules = Query.get(N)
    is2_files = earthaccess.open(granules)
    
    is2_times = []
    for u in is2_files:
        name = os.path.basename(u.url)
        is2_times.append(datetime.strptime(name[9:23], "%Y%m%d%H%M%S"))
    
    print("##### ICESat-2 files are loaded! #####")
    
    for k, f in enumerate(img_files[:]): #glob.glob("D:\\Floes\\S1_ground\\*.tif"):
        # os.rename(f, f.replace(" .tif", ".tif"))
    
        dh = 12 # hour difference between S1 and IS2
        
        basename = os.path.basename(f)
        t1 = datetime.strptime(basename[-19:-4], "%Y%m%dT%H%M%S") # - timedelta(seconds = 3600*dh)
        # t2 = datetime.strptime(basename[-19:-4], "%Y%m%dT%H%M%S") + timedelta(seconds = 3600*dh)

        imgname = f"D:\\Floes\\overlap_ground\\S1_{sub}_{basename[-19:-4]}+IS2.png"
    
        # if os.path.exists(imgname):
        start_time = datetime.strftime(t1, "%Y-%m-%d T%H:%M:%S")
        # end_time = datetime.strftime(t2, "%Y-%m-%d T%H:%M:%S")
    
        proceed = False
        
        for i, t in enumerate(is2_times):
            if abs(t - t1).total_seconds() < 3600*dh:
                proceed = True
                is2_file = is2_files[i]
    
        if proceed:

            if sub == "EW":
                dstCrs = {'init': 'EPSG:32723'}
            elif sub == "W":
                dstCrs = {'init': 'EPSG:32721'}
        
            ## Read Sentinel-1 SAR image using raterio ====================================
            img = rasterio.open(f)
            array = img.read(1)
            
            height = array.shape[0]
            width = array.shape[1]
            cols, rows = np.meshgrid(np.arange(width), np.arange(height))
        
            transform, width, height = calculate_default_transform(img.crs, dstCrs, img.width, img.height, *img.bounds)
            
            # XY projected coordinate of the image
            xs, ys = rasterio.transform.xy(transform, rows, cols)           
            
            xs = np.array(xs)
            ys = np.array(ys)
            # print(img.crs)
        
            img.close()
            # ===========================================================================
    
            print(start_time, basename, is2_file.url)
    
            ## ICESat-2 data using Earthaccess ===========================================
            df = read_ATL10(is2_file, bbox)
    
            if len(df) > 0:
    
                plt.figure(figsize = (8, 7), dpi = 200)
                plt.pcolormesh(xs, ys, array, cmap = "Greys_r", vmin = -0.5, vmax = -0.1)
                t = plt.annotate(f"{start_time}", xy=(0.03, 0.97), xycoords='axes fraction', va = "top", fontsize=12)
                t.set_bbox(dict(facecolor='white', alpha=0.5))
                plt.xlim(math.ceil(xs.min()/1000)*1000, math.floor(xs.max()/1000)*1000) #plt.xlim(420000, 540000)
                plt.ylim(math.ceil(ys.min()/1000)*1000, math.floor(ys.max()/1000)*1000) #plt.ylim(1440000, 1560000)
            
                # Convert pandas dataframe into geopandas geodataframe
                gdf = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.lon, df.lat))
                # Assign crs information into geodatafraem
                gdf.crs = "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"
                
                # Convert crs of the geodataframe into the crs of the Sentinel-2 image
                gdf = gdf.to_crs(dstCrs['init'])
        
                gdf['x'] = gdf.geometry.apply(lambda x: x.x)
                gdf['y'] = gdf.geometry.apply(lambda x: x.y)
                ## ============================================================================
        
                plt.scatter(gdf['x'], gdf['y'], c = gdf['fb'], s = 2, vmin = 0, vmax = 1, cmap = "jet")
                # imgname = imgname.replace(".png", "+IS2.png")
                
                plt.savefig(imgname, bbox_inches = "tight")
                
                plt.close()

print("END!")

QUEUEING TASKS | :   0%|          | 0/129 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/129 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/129 [00:00<?, ?it/s]

##### ICESat-2 files are loaded! #####
2021-01-02 T00:14:19 S1A_W_GRDH_1SSH_20210102T001419.tif https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2021/01/01/ATL10-02_20210101151012_01321001_006_01.h5
2021-01-05 T07:06:05 S1A_W_GRDH_1SSH_20210105T070605.tif https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2021/01/05/ATL10-02_20210105150152_01931001_006_01.h5
2021-02-03 T07:13:55 S1A_W_GRDH_1SSH_20210203T071355.tif https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2021/02/03/ATL10-02_20210203133757_06351001_006_01.h5
2021-02-08 T07:22:13 S1A_W_GRDH_1SSH_20210208T072213.tif https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2021/02/08/ATL10-02_20210208020357_07041001_006_01.h5
2021-02-12 T00:22:23 S1A_W_GRDH_1SSH_20210212T002223.tif https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2021/02/12/ATL10-02_

QUEUEING TASKS | :   0%|          | 0/122 [00:00<?, ?it/s]

...

##### ICESat-2 files are loaded! #####
2022-01-29 T07:14:07 S1A_W_GRDH_1SSH_20220129T071407.tif https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2022/01/28/ATL10-02_20220128202549_05741401_006_01.h5
2022-02-10 T00:47:00 S1A_W_GRDH_1SSH_20220210T004700.tif https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2022/02/10/ATL10-02_20220210083506_07651401_006_01.h5
2022-02-10 T07:14:07 S1A_W_GRDH_1SSH_20220210T071407.tif https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2022/02/10/ATL10-02_20220210083506_07651401_006_01.h5
2022-02-27 T07:22:18 S1A_W_GRDH_1SSH_20220227T072218.tif https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2022/02/27/ATL10-02_20220227073613_10241401_006_01.h5
2022-03-03 T00:22:28 S1A_W_GRDH_1SSH_20220303T002228.tif https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2022/03/03/ATL10-02_

QUEUEING TASKS | :   0%|          | 0/112 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/112 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/112 [00:00<?, ?it/s]

##### ICESat-2 files are loaded! #####
2023-03-01 T00:47:06 S1A_W_GRDH_1SSH_20230301T004706.tif https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2023/03/01/ATL10-02_20230301013253_10771801_006_02.h5
2023-03-01 T07:14:13 S1A_W_GRDH_1SSH_20230301T071413.tif https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2023/03/01/ATL10-02_20230301013253_10771801_006_02.h5
2023-03-13 T07:14:12 S1A_W_GRDH_1SSH_20230313T071412.tif https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2023/03/13/ATL10-02_20230313134214_12681801_006_02.h5
2023-03-17 T00:14:28 S1A_W_GRDH_1SSH_20230317T001428.tif https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2023/03/17/ATL10-02_20230317005940_13211801_006_02.h5
2023-03-30 T07:22:24 S1A_W_GRDH_1SSH_20230330T072224.tif https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL10/006/2023/03/30/ATL10-02_

In [27]:
df

[]

## Draw S1-alone image maps for the entire images

In [11]:
import gc
gc.collect()

283068

In [12]:
for year in [2019, 2020, 2021, 2022, 2023]:
    # sub = "W"
    img_files = glob.glob(f"D:\\Floes\\S1_ground\\S1A_{sub}_GRDH_1SSH_{year}*.tif")
    
    for k, f in tqdm(enumerate(img_files[:])): #glob.glob("D:\\Floes\\S1_ground\\*.tif"):
        # os.rename(f, f.replace(" .tif", ".tif"))
        
        basename = os.path.basename(f)
        t1 = datetime.strptime(basename[-19:-4], "%Y%m%dT%H%M%S") # - timedelta(seconds = 3600*dh)
        # t2 = datetime.strptime(basename[17:32], "%Y%m%dT%H%M%S") + timedelta(seconds = 3600*dh)
    
        imgname = f"D:\\Floes\\S1_ground_maps\\S1_map_{sub}_{basename[-19:-4]}.png"
    
        # if os.path.exists(imgname):
        start_time = datetime.strftime(t1, "%Y-%m-%d T%H:%M:%S")
        # end_time = datetime.strftime(t2, "%Y-%m-%d T%H:%M:%S")
    
        proceed = True
    
        if proceed:
        
            if sub == "EW":
                dstCrs = {'init': 'EPSG:32723'}
            elif sub == "W":
                dstCrs = {'init': 'EPSG:32721'}
        
            ## Read Sentinel-1 SAR image using raterio ====================================
            img = rasterio.open(f)
            array = img.read(1)
            
            height = array.shape[0]
            width = array.shape[1]
            cols, rows = np.meshgrid(np.arange(width), np.arange(height))
        
            transform, width, height = calculate_default_transform(img.crs, dstCrs, img.width, img.height, *img.bounds)
            
            # XY projected coordinate of the image
            xs, ys = rasterio.transform.xy(transform, rows, cols)           
            
            xs = np.array(xs)
            ys = np.array(ys)
            # print(img.crs)
        
            img.close()
            # ===========================================================================
    
            plt.figure(figsize = (8, 7), dpi = 150)
            plt.pcolormesh(xs, ys, array, cmap = "Greys_r", vmin = -0.5, vmax = -0.1)
            t = plt.annotate(f"{start_time}", xy=(0.03, 0.97), xycoords='axes fraction', va = "top", fontsize=12)
            t.set_bbox(dict(facecolor='white', alpha=0.5))
            plt.xlim(math.ceil(xs.min()/1000)*1000, math.floor(xs.max()/1000)*1000) #plt.xlim(420000, 540000)
            plt.ylim(math.ceil(ys.min()/1000)*1000, math.floor(ys.max()/1000)*1000) #plt.ylim(1440000, 1560000)
        
            plt.savefig(imgname, bbox_inches = "tight")
            
            plt.figure().clear();
            plt.close();
            plt.cla();
            plt.clf();
            
            del xs, ys, array
    
    print("END!")

191it [25:07,  7.89s/it]


END!


183it [23:40,  7.76s/it]


END!


155it [18:49,  7.29s/it]


...

<Figure size 1200x1050 with 0 Axes>

<Figure size 1200x1050 with 0 Axes>

<Figure size 1200x1050 with 0 Axes>

<Figure size 1200x1050 with 0 Axes>

<Figure size 1200x1050 with 0 Axes>

In [8]:
k

59

In [5]:
%matplotlib notebook
plt.plot([1,2])

<IPython.core.display.Javascript object>

In [17]:
Query = earthaccess.granule_query().concept_id('C2613553243-NSIDC_CPRD')\
.temporal('2019-01-01', '2023-12-31').bounding_box(bbox[0], bbox[1], bbox[2], bbox[3])

granules = Query.get()

In [9]:
import IPython


In [5]:
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots()
x = np.linspace(0, 2*np.pi, 100)
y = np.sin(3*x)
ax.plot(x, y)

<IPython.core.display.Javascript object>

In [93]:
img_files = glob.glob("D:\\Floes\\S1_ground\\*.tif")

for k, f in enumerate(img_files[312:]): #glob.glob("D:\\Floes\\S1_ground\\*.tif"):
    # os.rename(f, f.replace(" .tif", ".tif"))

    dh = 6
    basename = os.path.basename(f)
    t1 = datetime.strptime(basename[17:32], "%Y%m%dT%H%M%S") - timedelta(seconds = 3600*dh)
    t2 = datetime.strptime(basename[17:32], "%Y%m%dT%H%M%S") + timedelta(seconds = 3600*dh)

    start_time = datetime.strftime(t1, "%Y-%m-%d T%H:%M:%S")
    end_time = datetime.strftime(t2, "%Y-%m-%d T%H:%M:%S")